# Task 2 : Camera Calibration

In [52]:
#Imports
from scipy import linalg
import numpy as np

In [53]:
#Load World Values
with open("world.txt") as f:
    worldFullData = f.readlines()

#Creates worldmap with has rowsxline
worldmap = []
for worldline in worldFullData:
    worldmap.append(worldline.split("  ")[1:])

In [54]:
#Load Image Values
with open("image.txt") as f:
    imageFullData = f.readlines()

#Creates imagemap with has rowsxline
imagemap = []
for imageline in imageFullData:
    imagemap.append(imageline.split("  ")[1:])

In [55]:
#Matrix A
#20x12  n: 1->10
# [  Xn Yn Zn 1  0  0  0  0  -xnXn -xnYn -xn]
#    0  0  0  0  Xn Yn Zn 1  -ynXn -ynYn -yn]
#
#
matrix_a = [[0]*12 for x in xrange(20)]

for i in range(10):
    Xn = float(worldmap[0][i])
    Yn = float(worldmap[1][i])
    Zn = float(worldmap[2][i])

    xn = float(imagemap[0][i])
    yn = float(imagemap[1][i])


    matrix_a[2*i][0] = Xn
    matrix_a[2*i][1] = Yn
    matrix_a[2*i][2] = Zn
    matrix_a[2*i][3] = 1

    matrix_a[2*i+1][4] = Xn
    matrix_a[2*i+1][5] = Yn
    matrix_a[2*i+1][6] = Zn
    matrix_a[2*i+1][7] = 1

    matrix_a[2*i][8] = (-1*xn*Xn)
    matrix_a[2*i][9] = (-1*xn*Yn)
    matrix_a[2*i][10] = (-1*xn*Zn)
    matrix_a[2*i][11] = (-1*xn)

    matrix_a[2*i+1][8] = (-1*yn*Xn)
    matrix_a[2*i+1][9] = (-1*yn*Yn)
    matrix_a[2*i+1][10] = (-1*yn*Zn)
    matrix_a[2*i+1][11] = (-1*yn)

Compute P

In [56]:
#Calculate P by using SVD
U, eigen, Vh = linalg.svd(matrix_a)
Vh = Vh.transpose()
#Find Column with minimum
mincol = 0
minval = 500
for j in range(12):
    for i in range(12):
        if abs(Vh[j][i])<abs(minval):
            minval = Vh[j][i]
            mincol = i
P = Vh[:,mincol].reshape(3,4)
for m in P: 
    print P

[[  1.27000127e-01   2.54000254e-01   3.81000381e-01   5.08000508e-01]
 [  5.08000508e-01   3.81000381e-01   2.54000254e-01   1.27000127e-01]
 [  1.27000127e-01   5.17607385e-16   1.27000127e-01  -5.52857904e-16]]
[[  1.27000127e-01   2.54000254e-01   3.81000381e-01   5.08000508e-01]
 [  5.08000508e-01   3.81000381e-01   2.54000254e-01   1.27000127e-01]
 [  1.27000127e-01   5.17607385e-16   1.27000127e-01  -5.52857904e-16]]
[[  1.27000127e-01   2.54000254e-01   3.81000381e-01   5.08000508e-01]
 [  5.08000508e-01   3.81000381e-01   2.54000254e-01   1.27000127e-01]
 [  1.27000127e-01   5.17607385e-16   1.27000127e-01  -5.52857904e-16]]


In [57]:
#Verify answer
#

image_reproject = [[0]*10 for x in xrange(2)]

for idx in range(10):

    testmatworld = [worldmap[0][idx],worldmap[1][idx],worldmap[2][idx],1]

    Xres = (P[0][0]*float(testmatworld[0]))+(P[0][1]*float(testmatworld[1])) + (P[0][2]*float(testmatworld[2])) + (P[0][3]*float(testmatworld[3]))
    #print Xres

    Yres = (P[1][0]*float(testmatworld[0]))+(P[1][1]*float(testmatworld[1])) + (P[1][2]*float(testmatworld[2])) + (P[1][3]*float(testmatworld[3]))
    #print Yres

    Zres = (P[2][0]*float(testmatworld[0]))+(P[2][1]*float(testmatworld[1])) + (P[2][2]*float(testmatworld[2])) + (P[2][3]*float(testmatworld[3]))
    #print Zres

    image_reproject[0][idx] = Xres/Zres
    image_reproject[1][idx] = Yres/Zres
    
for j in range(len(image_reproject)):
    for i in range(len(image_reproject[j])):
        print image_reproject[j][0] - float(imagemap[j][0]) ,
    print ""

1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 
1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 1.7763568394e-15 


Compute Camera Center

In [58]:
for mp in P:
    print mp

[ 0.12700013  0.25400025  0.38100038  0.50800051]
[ 0.50800051  0.38100038  0.25400025  0.12700013]
[  1.27000127e-01   5.17607385e-16   1.27000127e-01  -5.52857904e-16]


In [59]:
Up,Sp,Vtp = linalg.svd(P)
Vp = Vtp.transpose()
Vp

array([[ 0.51135546, -0.67351259,  0.18680572, -0.5       ],
       [ 0.49037336, -0.1897408 , -0.68813691,  0.5       ],
       [ 0.50940459,  0.20160122,  0.67071895,  0.5       ],
       [ 0.48842249,  0.68537301, -0.20422368, -0.5       ]])

In [60]:
#Find Column with minimum
mincol2 = 0
minval2 = 500


In [61]:
for j2 in range(4):
    for i2 in range(4):
        if abs(Vp[j2][i2])<abs(minval2):
            minval2 = Vp[j2][i2]
            mincol2 = i2
C = Vp[:,mincol2].reshape(4,1)
for m2 in C: 
    print m2


[ 0.18680572]
[-0.68813691]
[ 0.67071895]
[-0.20422368]


In [62]:
#Get Camera Coordinates
#PC 
np.dot(P,C)

array([[ 0.00073585],
       [-0.02285667],
       [ 0.10890574]])

In [65]:
np.dot(worldmap,P)

TypeError: Cannot cast array data from dtype('float64') to dtype('S32') according to the rule 'safe'